In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, OneHotEncoder, MinMaxScaler, RobustScaler
import glob
import os
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from imblearn.over_sampling import SMOTE 
from imblearn.under_sampling import TomekLinks 
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
import xgboost as xgb
from sklearn.model_selection import RepeatedStratifiedKFold
import datetime

# Read Data

In [3]:
filePathTrain = "/content/drive/Shareddrives/ALDA_Project/ALDA/encodedDataTrainFitX4.csv"
filePathTest = "/content/drive/Shareddrives/ALDA_Project/ALDA/encodedDataTestFitX4.csv"
trainData = pd.read_csv(filePathTrain)
testData = pd.read_csv(filePathTest)

# OneHOtEncode Stations

In [6]:
dfTrain = pd.get_dummies(trainData.Station, prefix='station')
dfTest = pd.get_dummies(testData.Station, prefix='station')
encodedDataTrain = pd.concat([trainData, dfTrain], axis = 1)
encodedDataTrain = encodedDataTrain.drop(columns=["Station"], axis = 1)
encodedDataTest = pd.concat([testData, dfTest], axis = 1)
encodedDataTest = encodedDataTest.drop(columns=["Station"], axis = 1)

# Save data with OneHotEncoded Stations to csv

In [8]:
encodedDataTrain.to_csv("/content/drive/Shareddrives/ALDA_Project/ALDA/encodedDataTrainFitX4.csv", index=False)
encodedDataTest.to_csv("/content/drive/Shareddrives/ALDA_Project/ALDA/encodedDataTestFitX4.csv", index=False)